In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 1

In [2]:
from src.pipeline import *
# from src.convert_graphml_to_pyg_data_multithread import *

optuna.logging.set_verbosity(optuna.logging.WARNING)

%aimport src.pipeline
# %aimport src.convert_graphml_to_pyg_data_multithread

ModuleNotFoundError: No module named 'torch'

In [3]:
features = [
    "faces", "edges", "vertices", "quantity",
    "height", "width", "depth", "volume", "area",
    "bbox_height", "bbox_width", "bbox_depth", "bbox_volume",
    "bbox_area",
]
data = pd.read_csv("./data/synced_dataset_final.csv")

In [4]:
X = data[features]
X_train, X_test, y_train_index, y_test_index = train_test_split(
    X, range(len(X)),
    test_size=0.2,
    random_state=100,
    stratify=data["is_cnc"])
y_multi_train = data["multiclass_labels"].iloc[y_train_index]
y_multi_test = data["multiclass_labels"].iloc[y_test_index]

y_binary_train = data["is_cnc"].iloc[y_train_index]
y_binary_test = data["is_cnc"].iloc[y_test_index]

params = {
    "n_estimators": randint(100, 300),
    "max_depth": randint(3, 10),
    "learning_rate": uniform(0.005, 0.3),
}
best_search_binary = randomizedsearchcv_xgboost(
    X_train, y_binary_train,
    params, n_iter=50,
)

In [5]:
y_binary_pred = best_search_binary.best_estimator_.predict(X_test)
y_binary_prob = best_search_binary.best_estimator_.predict_proba(X_test)
y_binary_pred_train = best_search_binary.best_estimator_.predict(X_train)
y_binary_prob_train = best_search_binary.best_estimator_.predict_proba(X_train)
metrics_binary_validation = evaluate_classification(
    y_true=y_binary_test,
    y_pred=y_binary_pred,
    y_prob=y_binary_prob,
)
metrics_binary_train = evaluate_classification(
    y_true=y_binary_train,
    y_pred=y_binary_pred_train,
    y_prob=y_binary_prob_train,
)
pd.DataFrame([metrics_binary_train, metrics_binary_validation], index=["train", "validation"]).T

,train,validation
accuracy,0.951304,0.870498
f1_macro,0.947830,0.860209
f1_weighted,0.951724,0.871178
roc_auc,0.992262,0.936981
average_precision,0.995679,0.961874


In [6]:
feature_importance = best_search_binary.best_estimator_.feature_importances_

# If you have feature names
feature_names = X_train.columns  # assuming X_train is a DataFrame
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print(importance_df)

        feature  importance
7        volume    0.098656
12  bbox_volume    0.091828
3      quantity    0.087310
1         edges    0.086577
11   bbox_depth    0.084575
10   bbox_width    0.077502
9   bbox_height    0.074926
2      vertices    0.073620
5         width    0.069759
8          area    0.069744
4        height    0.065861
6         depth    0.060392
0         faces    0.059251
13    bbox_area    0.000000


In [8]:
fold_results = xgboost_optuna(
    data,
    n_trials=100,
    features=features
)

Optimizing fold 0:


  0%|          | 0/100 [00:00<?, ?it/s]

In [37]:
fold_results["fold_0"]["best_params"]

{'n_estimators': 229,
 'max_depth': 10,
 'learning_rate': 0.23010627398694375,
 'min_child_weight': 5,
 'subsample': 0.8515134589080573,
 'colsample_bytree': 0.7674160634634193,
 'reg_alpha': 3.8657297966420128,
 'reg_lambda': 0.6141160901516102}

In [28]:
fold_all_results = xgboost_optuna_run_with_best_params(data, fold_results["fold_0"]["best_params"], features=features)

Optimizing fold 0.0:
Optimizing fold 1.0:
Optimizing fold 2.0:
Optimizing fold 3.0:
Optimizing fold 4.0:
Optimizing fold 5.0:
Optimizing fold 6.0:
Optimizing fold 7.0:
Optimizing fold 8.0:
Optimizing fold 9.0:


In [33]:
pd.DataFrame(fold_all_results).T.to_csv(r"xgboost_optuna_results.csv")

In [40]:
results_df = pd.DataFrame(fold_all_results).T
results_df

,accuracy,f1_macro,f1_weighted,roc_auc,average_precision
fold_0.0,0.866720,0.857440,0.867966,0.936974,0.960162
fold_1.0,0.868489,0.858456,0.869361,0.934957,0.958854
fold_2.0,0.871383,0.861899,0.872372,0.937414,0.961699
fold_3.0,0.868489,0.858298,0.869295,0.932645,0.956106
fold_4.0,0.870096,0.860057,0.870892,0.932009,0.955883
fold_5.0,0.864309,0.854771,0.865529,0.935983,0.959806
fold_6.0,0.863505,0.853453,0.864548,0.932416,0.958994
fold_7.0,0.861576,0.851569,0.862709,0.931229,0.955024
fold_8.0,0.864930,0.855317,0.866105,0.929549,0.956050
fold_9.0,0.864608,0.854895,0.865755,0.933825,0.958898


In [58]:
xg_auroc = results_df["roc_auc"].tolist()
xg_auroc

[0.9369739113754075,
 0.9349566694124418,
 0.9374142697554756,
 0.9326450277022377,
 0.9320086454053684,
 0.935982573808659,
 0.9324164794747773,
 0.9312290787609381,
 0.9295489560254133,
 0.9338254199544237]

In [44]:
from scipy.stats import sem
sem(results_df["roc_auc"]), np.mean(results_df["roc_auc"])

(0.0008183796760417202, 0.9337001031675143)

In [48]:
import json
with open(r"C:\Users\phanm\OneDrive\Documents\Study\PythonProjects\manufacturing-tech-gnn\optuna_gine_all_fold_results.json", "rb") as f:
    all_fold_results = json.load(f)

In [57]:
gnn_auroc = [item["val_tracker"]["auroc"] for item in all_fold_results.values()]
sem(gnn_auroc), np.mean(gnn_auroc)

(0.0007362313565860531, 0.9425765573978424)

In [59]:
gnn_auroc

[0.9475449919700623,
 0.9430391788482666,
 0.944582998752594,
 0.9434629678726196,
 0.9422585368156433,
 0.9399271011352539,
 0.9431166648864746,
 0.9405849575996399,
 0.9404380917549133,
 0.9408100843429565]

In [60]:
from scipy.stats import ttest_rel
t_stat, p_value = ttest_rel(gnn_auroc, xg_auroc)

print("t-statistic:", t_stat)
print("p-value:", p_value)

t-statistic: 12.20764749791187
p-value: 6.652107671946196e-07
